<a href="https://colab.research.google.com/github/shinzokuro/YOLO_utils/blob/main/train_YOLOv5_template_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# install dependencies

In [ ]:
!git clone https://github.com/ultralytics/yolov5
!git clone https://github.com/shinzokuro/YOLO_utils.git
%pip install -r yolov5/requirements.txt
%pip install wandb

Cloning into 'yolov5'...
remote: Enumerating objects: 17067, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 17067 (delta 24), reused 28 (delta 12), pack-reused 17022 (from 1)
Receiving objects: 100% (17067/17067), 15.68 MiB | 31.79 MiB/s, done.
Resolving deltas: 100% (11714/11714), done.
Cloning into 'YOLO_utils'...
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 145 (delta 70), reused 112 (delta 43), pack-reused 0 (from 0)
Receiving objects: 100% (145/145), 26.80 KiB | 2.98 MiB/s, done.
Resolving deltas: 100% (70/70), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.5/898.5 kB 14.9 MB/s eta 0:00:00


In [ ]:
import os
import wandb
from pathlib import Path
from google.colab import userdata
os.environ['WANDB_API_KEY'] = userdata.get('KURO_KEY')
ROOT_DRIVE_DIR = Path(userdata.get('KURO_DIRECTORY'))
TEST_DATA_DIRECTORY = Path(userdata.get('TEST_DATA_DIRECTORY'))
from yolov5 import train, val
from YOLO_utils.scripts.collab.workspace import CollabWorkspaceManager, ExperimentTracker
from YOLO_utils.scripts.utils.ziputils import get_zip_path
from YOLO_utils.scripts.YOLOv5.config import create_val_config, create_train_config
from YOLO_utils.scripts.utils.yaml_utils import read_yaml, create_dataset_yamlfile
from YOLO_utils.scripts.utils.yaml_utils import create_val_yaml
from YOLO_utils.scripts.YOLOv5.plot_patch import setup_patch
setup_patch()

In [ ]:
ROOT_DRIVE_DIR = Path(r"/content/drive/MyDrive/work/kuro_workspace")
priority_path = ROOT_DRIVE_DIR / "train_priority_list.txt"
tracker = ExperimentTracker(priority_path)
current_expirement = tracker.get_experiment_to_run()
if current_expirement is None:
    raise Exception("no experiments to run")
else:
    print(f"running experiment {current_expirement}")
experiment_directory_drive = ROOT_DRIVE_DIR / current_expirement
dataset_zip_file = get_zip_path(experiment_directory_drive)


# setup collab workspace

In [ ]:
workspace = CollabWorkspaceManager(current_expirement)
workspace.setup(dataset_zip_file,TEST_DATA_DIRECTORY)

# setup training parameters

In [ ]:
dataset_directory = workspace.dataset / current_expirement
create_dataset_yamlfile(dataset_directory)
train_config = create_train_config(read_yaml(dataset_directory / "train.yaml"), workspace.dataset, workspace.model, current_expirement)
train.main(train_config)

# test data evaluation

In [ ]:
def main(save_path: Path, test_data_path, pt_files: dict):
    test_data = [directory for directory in Path(test_data_path).iterdir() if directory.is_dir()]
    for data_path in test_data:
        data_yaml = create_val_yaml(data_path)
        project_path = save_path / data_path.stem
        for model_name, pt_file_path in pt_files.items():
            opt = create_val_config(
                weights_path=pt_file_path,
                data_path=data_yaml,
                project_path=project_path,
                name=model_name,
            )
            val.main(opt)

In [ ]:
pt_files = {
    train_config.name :  workspace.model / train_config.project / train_config.name / "weights" / "best.pt"
}
main(workspace.results, workspace.test_data, pt_files)

In [ ]:
tracker._mark_expirement_as_completed()
workspace.save_results(experiment_directory_drive)